# 目的
- Qwen2.5-32B-Instruct-AWQをSFTTrainerする
- apply_templateを使う。

ref:
- https://qwen.readthedocs.io/en/v1.5/training/SFT/example.html
- https://huggingface.co/docs/trl/v0.12.0/en/sft_trainer#trl.SFTTrainer

ref(old):
- https://github.com/unslothai/unsloth
- https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing#scrollTo=6bZsfBuZDeCL
- https://huggingface.co/blog/mlabonne/sft-llama3
- https://huggingface.co/docs/trl/sft_trainer#accelerate-fine-tuning-2x-using-unsloth

In [1]:
# path setting
EXP_NAME = "e028-causal"
MODEL_NAME = "Qwen/Qwen2.5-32B-Instruct-AWQ"

COMPETITION_NAME = "eedi-mining-misconceptions-in-mathematics"

DATA_PATH = "data"
DATASET_NAME = f"{EXP_NAME}-{MODEL_NAME.split('/')[-1].replace('.', '-')}"
OUTPUT_PATH = f"output/causal/{EXP_NAME}"
MODEL_OUTPUT_PATH = f"{OUTPUT_PATH}/trained_model"
ENV_PATH = "env_file"

# experiment parameter
DEBUG = False
TRAINING = False
UPLOAD_DATA_TO_S3 = True
UPLOAD_DATA_TO_KAGGLE = True
REMOVE_LOCAL_FILE = False
WANDB = True

# USE_FOLD = 0
# USE_DATA_RATE = 1.0
# VALID_DATA_SIZE = 3000

# model parameter
TRAINING_MAX_LENGTH = 512
SEED = 42
EPOCH = 5
LR = 2e-04
TRAIN_BS = 1  # 2  # 4  # 16
GRAD_ACC_STEP = 128 // TRAIN_BS  # 仮想的なバッチサイズはTRAIN_BS * GRAD_ACC_STEPとなる
EVAL_BS = 1  # 2  # 4  # 16
# NUM_LABELS = 3

FREEZE_LAYERS = 16

# rola parameter
LORA_R = 16
LORA_ALPHA = LORA_R * 2
LORA_BIAS = "none"

RESUME_FROM_CHECKPOINT = False  # 途中から再開する場合はTrueにする

In [2]:
!nvidia-smi

Mon Nov  4 01:10:04 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.107.02             Driver Version: 550.107.02     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        On  |   00000000:0B:00.0 Off |                  N/A |
|  0%   46C    P8             20W /  315W |       2MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!python --version

Python 3.10.14


In [4]:
def resolve_path(base_path: str) -> str:
    import os

    cwd = os.getcwd()
    print(cwd)
    if cwd == f"/notebooks":
        print("Jupyter Kernel By VSCode!")
        return f"/notebooks/{COMPETITION_NAME}/{base_path}"
    elif cwd == f"/notebooks/{COMPETITION_NAME}":
        print("nohup!")
        return base_path
    elif cwd == f"/notebooks/{COMPETITION_NAME}/{COMPETITION_NAME}/exp":
        print("Jupyter Lab!")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/reranker":
        print("VastAi! Reranker")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/retriever":
        print("VastAi! Retriever")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/causal":
        print("VastAi! Causal")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}":
        print("VastAi!")
        return base_path
    else:
        raise Exception("Unknown environment")


DATA_PATH = resolve_path(DATA_PATH)
print(DATA_PATH)
OUTPUT_PATH = resolve_path(OUTPUT_PATH)
print(OUTPUT_PATH)
MODEL_OUTPUT_PATH = resolve_path(MODEL_OUTPUT_PATH)
print(MODEL_OUTPUT_PATH)
ENV_PATH = resolve_path(ENV_PATH)
print(ENV_PATH)

/root/eedi-mining-misconceptions-in-mathematics/exp/causal
VastAi! Causal
../../data
/root/eedi-mining-misconceptions-in-mathematics/exp/causal
VastAi! Causal
../../output/causal/e028-causal
/root/eedi-mining-misconceptions-in-mathematics/exp/causal
VastAi! Causal
../../output/causal/e028-causal/trained_model
/root/eedi-mining-misconceptions-in-mathematics/exp/causal
VastAi! Causal
../../env_file


In [5]:
def validate_dataset_name(dataset_name: str) -> None:
    if len(dataset_name) < 6 or len(dataset_name) > 50:
        raise Exception(
            f"データセットの文字列は6~50文字にしてください。現在{len(DATASET_NAME)}文字"
        )
    if "_" in dataset_name or "." in dataset_name:
        raise Exception("datasetの名称に'_'、'.'の使用は禁止です")


validate_dataset_name(DATASET_NAME)

# import

In [6]:
import os
import random

import polars as pl
import numpy as np

import wandb

import torch
from datasets import Dataset
from transformers import TrainingArguments, AutoModelForCausalLM, AutoTokenizer

from peft import LoraConfig
from trl import SFTConfig, SFTTrainer
# from unsloth import FastLanguageModel, is_bfloat16_supported

In [7]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
NUM_PROC = 16  # os.cpu_count()

In [8]:
# import importlib.metadata

import torch
import transformers
import datasets
import trl
import peft


assert torch.__version__ == "2.3.1+cu121"
assert transformers.__version__ == "4.44.2"
# assert transformers.__version__ == "4.46.1" # unslothがimportできない
assert datasets.__version__ == "3.0.0"
assert trl.__version__ == "0.11.4"
# assert importlib.metadata.version("unsloth") == "2024.10.2"
assert peft.__version__ == "0.13.2"

In [9]:
# Seed the same seed to all
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(SEED)

In [10]:
from dotenv import load_dotenv

load_dotenv(f"{ENV_PATH}/.env")

True

# Wandb

In [11]:
if WANDB:
    wandb.login(key=os.environ["WANDB_API_KEY"])
    wandb.init(project=COMPETITION_NAME, name=EXP_NAME)
    REPORT_TO = "wandb"
else:
    REPORT_TO = "none"

REPORT_TO

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: sinchir0. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


'wandb'

# Model Tokenizer Load

In [12]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    padding_side="right",
    use_fast=False,
)

In [13]:
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name=MODEL_NAME,
#     max_seq_length=TRAINING_MAX_LENGTH,
#     dtype=None,  # 学習のタイミングでRuntimeError: Expected out tensor to have dtype c10::Half, but got float insteadと出てしまう
#     # dtype=torch.float16,
#     # load_in_4bit=True,
# )

In [14]:
# https://www.kaggle.com/code/kishanvavdara/lmsys-llama-3-tpu-train
# tokenizer.padding_side = "right"

# Data Import & Preprocess

In [15]:
train_long = pl.read_csv(f"{DATA_PATH}/train_long.csv").filter(
    (pl.col("CorrectAnswer") != pl.col("AnswerAlphabet"))
    & (pl.col("MisconceptionId").is_not_null())
)

In [16]:
if DEBUG:
    train_long = train_long.head(100)

In [17]:
train_long.head()

QuestionId,ConstructName,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,AnswerType,AnswerText,AllText,AnswerAlphabet,QuestionId_Answer,MisconceptionId,MisconceptionName
i64,str,str,str,str,str,str,str,str,str,str,i64,str
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""\( 3 \times(2+4)-5 \)""","""AnswerDText""","""Does not need brackets""","""Use the order of operations to…","""D""","""0_D""",1672,"""Confuses the order of operatio…"
1000,"""Simplify an algebraic fraction…","""Simplifying Algebraic Fraction…","""Simplify the following, if pos…","""B""","""\( -1 \)""","""AnswerAText""","""\( t \)""","""Simplify an algebraic fraction…","""A""","""1000_A""",891,"""Incorrectly cancels what they …"
1000,"""Simplify an algebraic fraction…","""Simplifying Algebraic Fraction…","""Simplify the following, if pos…","""B""","""\( -1 \)""","""AnswerCText""","""\( 1 \)""","""Simplify an algebraic fraction…","""C""","""1000_C""",891,"""Incorrectly cancels what they …"
1000,"""Simplify an algebraic fraction…","""Simplifying Algebraic Fraction…","""Simplify the following, if pos…","""B""","""\( -1 \)""","""AnswerDText""","""Does not simplify""","""Simplify an algebraic fraction…","""D""","""1000_D""",353,"""Does not recognise when one pa…"
1001,"""Round numbers to two decimal p…","""Rounding to Decimal Places""","""What is \( \mathbf{3 . 5 1 6 3…","""B""","""\( 3.52 \)""","""AnswerAText""","""\( 3.51 \)""","""Round numbers to two decimal p…","""A""","""1001_A""",1379,"""Rounds down instead of up"""


In [18]:
QUESTION_TEMPLATE = """Question: {question_text}
Incorrect Answer: {incorrect_answer_text}
Correct Answer: {correct_answer_text}
Construct Name: {construct_name}
Subject Name: {subject_name}"""


class ApplyTemplateForTrain:
    def make_masaages(self, question: str, answer: str) -> list[dict[str, str]]:
        return [
            {
                "role": "system",
                "content": "You are a mathematics expert. You are given a mathematics question, an incorrect answer, a correct answer, a construct name, and a subject name. Using these, identify what misconception exists in the incorrect answer.",
            },
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer},
        ]

    def apply_template(self, examples):
        messages = []
        for (
            question_text,
            incorrect_answer_text,
            correct_answer_text,
            construct_name,
            subject_name,
            misconception_name,
        ) in zip(
            examples["QuestionText"],
            examples["AnswerText"],  # incorrect_answer_text
            examples["CorrectAnswerText"],
            examples["ConstructName"],
            examples["SubjectName"],
            examples["MisconceptionName"],
        ):
            message = self.make_masaages(
                question=QUESTION_TEMPLATE.format(
                    question_text=question_text,
                    incorrect_answer_text=incorrect_answer_text,
                    correct_answer_text=correct_answer_text,
                    construct_name=construct_name,
                    subject_name=subject_name,
                ),
                answer=misconception_name,
            )
            messages.append(message)

        text = [
            tokenizer.apply_chat_template(
                message,
                tokenize=False,
                add_generation_prompt=False,
            )
            for message in messages
        ]

        return {"text": text}


train = Dataset.from_polars(train_long).map(
    ApplyTemplateForTrain().apply_template, batched=True
)

Map:   0%|          | 0/4370 [00:00<?, ? examples/s]

In [19]:
print(train[0]["text"])

<|im_start|>system
You are a mathematics expert. You are given a mathematics question, an incorrect answer, a correct answer, a construct name, and a subject name. Using these, identify what misconception exists in the incorrect answer.<|im_end|>
<|im_start|>user
Question: \[
3 \times 2+4-5
\]
Where do the brackets need to go to make the answer equal \( 13 \) ?
Incorrect Answer: Does not need brackets
Correct Answer: \( 3 \times(2+4)-5 \)
Construct Name: Use the order of operations to carry out calculations involving powers
Subject Name: BIDMAS<|im_end|>
<|im_start|>assistant
Confuses the order of operations, believes addition comes before multiplication <|im_end|>



In [20]:
print(train[10]["text"])

<|im_start|>system
You are a mathematics expert. You are given a mathematics question, an incorrect answer, a correct answer, a construct name, and a subject name. Using these, identify what misconception exists in the incorrect answer.<|im_end|>
<|im_start|>user
Question: This pictogram shows the different types of music Bob has in his music collection.

How many Blues CDs does Bob have? ![A pictogram showing the number of CDs Bob has in his musical collection. Pop has 3 and a half symbols, rock has 2 symbols, blues has 2 and a quarter symbols, jazz has 3 and a quarter symbols and classical has 1 and three-quarter symbols. Each symbol represents 4 CDs.]()
Incorrect Answer: \( 2.25 \)
Correct Answer: \( 9 \)
Construct Name: Interpret a pictogram involving fractions of symbols
Subject Name: Pictogram<|im_end|>
<|im_start|>assistant
When interpreting a pictogram, thinks each symbol stands for 1<|im_end|>



# Split Train Valid

In [21]:
train, valid = (
    train.filter(lambda x: x["QuestionId"] % 3 != 0, num_proc=NUM_PROC),
    train.filter(lambda x: x["QuestionId"] % 3 == 0, num_proc=NUM_PROC),
)

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Filter (num_proc=16):   0%|          | 0/4370 [00:00<?, ? examples/s]

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Filter (num_proc=16):   0%|          | 0/4370 [00:00<?, ? examples/s]

# Model

In [22]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16)
model.to("cuda")

You have loaded an AWQ model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model.


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 5120)
    (layers): ModuleList(
      (0-63): 64 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): WQLinear_GEMM(in_features=5120, out_features=5120, bias=True, w_bit=4, group_size=128)
          (k_proj): WQLinear_GEMM(in_features=5120, out_features=1024, bias=True, w_bit=4, group_size=128)
          (v_proj): WQLinear_GEMM(in_features=5120, out_features=1024, bias=True, w_bit=4, group_size=128)
          (o_proj): WQLinear_GEMM(in_features=5120, out_features=5120, bias=False, w_bit=4, group_size=128)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): WQLinear_GEMM(in_features=5120, out_features=27648, bias=False, w_bit=4, group_size=128)
          (up_proj): WQLinear_GEMM(in_features=5120, out_features=27648, bias=False, w_bit=4, group_size=128)
          (down_proj): WQLinear_GEMM(in_features=27648, out_features=5120, bi

In [23]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 5120)
    (layers): ModuleList(
      (0-63): 64 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): WQLinear_GEMM(in_features=5120, out_features=5120, bias=True, w_bit=4, group_size=128)
          (k_proj): WQLinear_GEMM(in_features=5120, out_features=1024, bias=True, w_bit=4, group_size=128)
          (v_proj): WQLinear_GEMM(in_features=5120, out_features=1024, bias=True, w_bit=4, group_size=128)
          (o_proj): WQLinear_GEMM(in_features=5120, out_features=5120, bias=False, w_bit=4, group_size=128)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): WQLinear_GEMM(in_features=5120, out_features=27648, bias=False, w_bit=4, group_size=128)
          (up_proj): WQLinear_GEMM(in_features=5120, out_features=27648, bias=False, w_bit=4, group_size=128)
          (down_proj): WQLinear_GEMM(in_features=27648, out_features=5120, bi

# Model Predict Check Before Training

In [24]:
class ApplyTemplateForPredict:
    def make_masaages(self, question: str) -> list[dict[str, str]]:
        return [
            {
                "role": "system",
                "content": "You are a mathematics expert. You are given a mathematics question, an incorrect answer, a correct answer, a construct name, and a subject name. Using these, identify what misconception exists in the incorrect answer.",
            },
            {"role": "user", "content": question},
        ]

    def apply_template(self, examples):
        messages = []
        for (
            question_text,
            incorrect_answer_text,
            correct_answer_text,
            construct_name,
            subject_name,
        ) in zip(
            examples["QuestionText"],
            examples["AnswerText"],  # incorrect_answer_text
            examples["CorrectAnswerText"],
            examples["ConstructName"],
            examples["SubjectName"],
        ):
            message = self.make_masaages(
                question=QUESTION_TEMPLATE.format(
                    question_text=question_text,
                    incorrect_answer_text=incorrect_answer_text,
                    correct_answer_text=correct_answer_text,
                    construct_name=construct_name,
                    subject_name=subject_name,
                )
            )
            messages.append(message)

        text = [
            tokenizer.apply_chat_template(
                message, tokenize=False, add_generation_prompt=True
            )
            for message in messages
        ]

        return {"text": text}


valid_for_predict = valid.map(ApplyTemplateForPredict().apply_template, batched=True)

Map:   0%|          | 0/1460 [00:00<?, ? examples/s]

In [25]:
def predict(valid: Dataset, idx: int, model):
    inputs = tokenizer(
        valid["text"][idx],
        return_tensors="pt",
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=64, do_sample=True)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
    print("\n### Answer Misconception Name:")
    print(valid["MisconceptionName"][idx])

In [26]:
predict(valid_for_predict, 0, model)

system
You are a mathematics expert. You are given a mathematics question, an incorrect answer, a correct answer, a construct name, and a subject name. Using these, identify what misconception exists in the incorrect answer.
user
Question: \[
3 \times 2+4-5
\]
Where do the brackets need to go to make the answer equal \( 13 \) ?
Incorrect Answer: Does not need brackets
Correct Answer: \( 3 \times(2+4)-5 \)
Construct Name: Use the order of operations to carry out calculations involving powers
Subject Name: BIDMAS
assistant
The misconception in the incorrect answer is that it fails to recognize the necessity of using brackets to alter the order of operations (BIDMAS/BODMAS rules) to achieve the desired result. The student may have assumed that the expression can yield the correct answer without any modification or addition of brackets, which is incorrect. In

### Answer Misconception Name:
Confuses the order of operations, believes addition comes before multiplication 


In [27]:
predict(valid_for_predict, 1, model)

system
You are a mathematics expert. You are given a mathematics question, an incorrect answer, a correct answer, a construct name, and a subject name. Using these, identify what misconception exists in the incorrect answer.
user
Question: Convert \( \frac{3}{20} \) into a decimal
Incorrect Answer: \( 0.3 \)
Correct Answer: \( 0.15 \)
Construct Name: Convert fractions less than 1 to terminating decimals of 2 decimal places
Subject Name: Converting between Fractions and Decimals
assistant
The misconception in the incorrect answer is that the student has not correctly understood how to convert a fraction to its decimal form. Specifically, they seem to have incorrectly placed the decimal point. The student may have simply taken the numerator and placed it after a decimal point without considering the value of the denominator. 

In this case,

### Answer Misconception Name:
Converts a fraction to a decimal by using only the numerator after the decimal point


In [28]:
predict(valid_for_predict, 2, model)

system
You are a mathematics expert. You are given a mathematics question, an incorrect answer, a correct answer, a construct name, and a subject name. Using these, identify what misconception exists in the incorrect answer.
user
Question: Convert \( \frac{3}{20} \) into a decimal
Incorrect Answer: \( 3.20 \)
Correct Answer: \( 0.15 \)
Construct Name: Convert fractions less than 1 to terminating decimals of 2 decimal places
Subject Name: Converting between Fractions and Decimals
assistant
The misconception in the incorrect answer lies in a fundamental misunderstanding of how to convert a fraction into a decimal. The student has incorrectly interpreted the fraction \( \frac{3}{20} \) as the decimal number \( 3.20 \), which suggests they may be confusing the numerator and denominator with whole numbers

### Answer Misconception Name:
Converts a fraction to a decimal by placing a decimal point between the numerator and denominator


# Model

In [29]:
# model = FastLanguageModel.get_peft_model(
#     model,
#     r=LORA_R,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
#     target_modules=[
#         "q_proj",
#         "k_proj",
#         "v_proj",
#         # "o_proj",
#         # "gate_proj",
#         # "up_proj",
#         # "down_proj",
#         # "embed_tokens",
#         # "lm_head",
#     ],
#     lora_alpha=LORA_ALPHA,
#     lora_dropout=0,  # Supports any, but = 0 is optimized
#     bias="none",  # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
#     random_state=SEED,
#     use_rslora=False,  # We support rank stabilized LoRA
#     loftq_config=None,  # And LoftQ
#     layers_to_transform=[i for i in range(32) if i >= FREEZE_LAYERS],
# )

# trainer = SFTTrainer(
#     MODEL_NAME,
#     train_dataset=train,
#     eval_dataset=valid,
#     args=SFTConfig(output_dir="/tmp"),
#     peft_config=peft_config,
#     dataset_text_field="text"
# )

In [30]:
# training_arguments = TrainingArguments(
training_arguments = SFTConfig(
    output_dir=MODEL_OUTPUT_PATH,
    learning_rate=LR,
    per_device_train_batch_size=TRAIN_BS,
    gradient_accumulation_steps=GRAD_ACC_STEP,
    eval_accumulation_steps=GRAD_ACC_STEP,
    per_device_eval_batch_size=EVAL_BS,
    num_train_epochs=EPOCH,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=0.1,
    save_strategy="steps",
    save_steps=0.1,
    save_total_limit=1,
    logging_steps=2,
    seed=SEED,
    metric_for_best_model="eval_loss",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine_with_restarts",
    report_to=REPORT_TO,
    run_name=EXP_NAME,
    load_best_model_at_end=True,
    fp16=True,
    fp16_full_eval=True,
    gradient_checkpointing=True,
    remove_unused_columns=False,
    group_by_length=True,
)

In [31]:
peft_config = LoraConfig(
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        # "o_proj",
        # "gate_proj",
        # "up_proj",
        # "down_proj",
        # "embed_tokens",
        # "lm_head",
    ],
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [32]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train,
    eval_dataset=valid,
    dataset_text_field="text",
    max_seq_length=TRAINING_MAX_LENGTH,
    dataset_num_proc=NUM_PROC,
    # packing=False,  # Can make training 5x faster for short sequences.
    packing=True,  # Can make training 5x faster for short sequences.
    args=training_arguments,
    peft_config=peft_config,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_num_proc, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:195: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:297: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you pa

In [33]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3090. Max memory = 23.682 GB.
19.566 GB of memory reserved.


In [37]:
if TRAINING:
    trainer.train()
else:
    from peft import AutoPeftModelForCausalLM

    model = AutoPeftModelForCausalLM.from_pretrained(
        MODEL_OUTPUT_PATH + "/checkpoint-32", torch_dtype=torch.float16
    )
    model.to("cuda")

    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME,
        padding_side="right",
        use_fast=False,
    )

    # model, tokenizer = FastLanguageModel.from_pretrained(
    #     model_name=MODEL_OUTPUT_PATH + "/lora_model",
    #     max_seq_length=TRAINING_MAX_LENGTH,
    #     dtype=None,  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    #     # load_in_4bit=True,  # Use 4bit quantization to reduce memory usage. Can be False.
    # )

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 68.00 MiB. GPU 

In [35]:
# if TRAINING:
#     trainer_stats = trainer.train()
# else:
#     model, tokenizer = FastLanguageModel.from_pretrained(
#         model_name=MODEL_OUTPUT_PATH + "/lora_model",
#         max_seq_length=TRAINING_MAX_LENGTH,
#         dtype=None,  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
#         # load_in_4bit=True,  # Use 4bit quantization to reduce memory usage. Can be False.
#     )

# Predict Valid

In [36]:
predict(valid_for_predict, 0, model)

In [37]:
predict(valid_for_predict, 1, model)

In [38]:
predict(valid_for_predict, 2, model)

# Save

In [39]:
model.save_pretrained(f"{MODEL_OUTPUT_PATH}/lora_model")  # Local saving
tokenizer.save_pretrained(f"{MODEL_OUTPUT_PATH}/lora_model")

RuntimeError: The weights trying to be saved contained shared tensors [{'model.layers.0.self_attn.q_proj.quant_linear_module.qweight', 'model.layers.0.self_attn.q_proj.base_layer.qweight'}, {'model.layers.0.self_attn.q_proj.quant_linear_module.qzeros', 'model.layers.0.self_attn.q_proj.base_layer.qzeros'}, {'model.layers.0.self_attn.q_proj.base_layer.scales', 'model.layers.0.self_attn.q_proj.quant_linear_module.scales'}, {'model.layers.0.self_attn.q_proj.base_layer.bias', 'model.layers.0.self_attn.q_proj.quant_linear_module.bias'}, {'model.layers.0.self_attn.k_proj.quant_linear_module.qweight', 'model.layers.0.self_attn.k_proj.base_layer.qweight'}, {'model.layers.0.self_attn.k_proj.base_layer.qzeros', 'model.layers.0.self_attn.k_proj.quant_linear_module.qzeros'}, {'model.layers.0.self_attn.k_proj.base_layer.scales', 'model.layers.0.self_attn.k_proj.quant_linear_module.scales'}, {'model.layers.0.self_attn.k_proj.quant_linear_module.bias', 'model.layers.0.self_attn.k_proj.base_layer.bias'

# AWSへのアップロード

In [40]:
# S3へのアップロード
if not DEBUG and UPLOAD_DATA_TO_S3:
    # uninstall
    !sudo rm /usr/bin/aws
    !sudo rm /usr/bin/aws_completer
    !sudo rm -rf /usr/local/aws-cli

    # install
    !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
    !unzip -o -qq awscliv2.zip
    !sudo ./aws/install --update

    # upload
    output_name = MODEL_OUTPUT_PATH.split("/")[-1]
    os.system(
        f"aws s3 cp --recursive {MODEL_OUTPUT_PATH} s3://{COMPETITION_NAME}/trained_model/{output_name}"
    )

In [41]:
# ダウンロード（参考）
# !sudo rm /usr/bin/aws
# !sudo rm /usr/bin/aws_completer
# !sudo rm -rf /usr/local/aws-cli

# !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
# !unzip -o -qq awscliv2.zip
# !sudo ./aws/install --update

# !aws s3 cp --recursive s3://automated-essay-scoring/trained_model/e005-regression /notebooks/automated_essay_scoring/trained_models/e005-regression

# Kaggle Datasetへのupload

In [38]:
import os

os.system("mkdir -p ~/.kaggle/")
os.system(f"cp {ENV_PATH}/kaggle.json ~/.kaggle/")
os.system("chmod 600 ~/.kaggle/kaggle.json")

0

In [39]:
if not DEBUG and UPLOAD_DATA_TO_KAGGLE:
    import os
    import json

    from kaggle.api.kaggle_api_extended import KaggleApi

    def dataset_create_new(dataset_name: str, upload_dir: str):
        dataset_metadata = {}
        dataset_metadata["id"] = f"sinchir0/{dataset_name}"
        dataset_metadata["licenses"] = [{"name": "CC0-1.0"}]
        dataset_metadata["title"] = dataset_name
        with open(os.path.join(upload_dir, "dataset-metadata.json"), "w") as f:
            json.dump(dataset_metadata, f, indent=4)
        api = KaggleApi()
        api.authenticate()
        api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode="tar")

    print(f"Create Dataset name:{DATASET_NAME}, output_dir:{MODEL_OUTPUT_PATH}")
    dataset_create_new(dataset_name=DATASET_NAME, upload_dir=MODEL_OUTPUT_PATH)

Create Dataset name:e028-causal-Qwen2-5-32B-Instruct-AWQ, output_dir:../../output/causal/e028-causal/trained_model
Starting upload for file checkpoint-32.tar


100%|██████████| 269M/269M [01:26<00:00, 3.27MB/s] 


Upload successful: checkpoint-32.tar (269MB)
Starting upload for file lora_model.tar


100%|██████████| 10.0k/10.0k [00:00<00:00, 15.6kB/s]


Upload successful: lora_model.tar (10KB)


# ローカルからのデータの削除

In [44]:
# if not DEBUG and REMOVE_LOCAL_FILE:
#     # ローカルからは削除
#     os.system(f"rm -rf {MODEL_OUTPUT_PATH}")

In [45]:
if WANDB:
    wandb.finish()

In [46]:
print("finish Notebook!")